##  Sentinel-3 OLCI Level-2 Tiling

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-3 OLCI level-2 tiling'),
                ('abstract', 'This service takes as input one or more Sentinel-3 OLCI Level 2 (OL_2_LFR____) products and tiles'),
                ('identifier', 'ewf-wfp-03-01-01')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [2]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 OLCI Level 2 (OL_2_LFR____)'),
                        ('abstract', 'Sentinel-3 OLCI Level 2 (OL_2_LFR____) catalogue reference'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?uid=S3A_OL_2_LFR____20190101T080250_20190101T080550_20190102T125100_0179_039_363_2880_LN1_O_NT_002'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

In [3]:
tiling_level = dict([('identifier', 'tiling_level'),
                ('value', '5'),
                ('title', 'Tiling level'),
                ('abstract', 'Tiling level'),
                ('max_occurs', '1')])

**Data path**

This path defines where the data is staged-in. 

In [4]:
data_path = '/workspace/data/s3'

In [5]:
input_catalog = '/workspace/data/s3/catalog.json'

### Workflow

#### Import the packages

In [6]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'

In [7]:
import os
import sys
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
sys.path.append('.')
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType
from tiling import s3_tiles
import glob
import numpy as np
gdal.UseExceptions()

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [10]:
collection = next(cat.get_children())

In [11]:
item = next(collection.get_items())

In [12]:
item

<EOItem id=S3A_OL_2_LFR____20190101T080250_20190101T080550_20190102T125100_0179_039_363_2880_LN1_O_NT_002>

### Import Sentinel-3 SLSTR product

In [13]:
operators = ['Read', 
             'Reproject',
             'Write']

In [14]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_olci'

In [15]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Reproject=reproject,
           Write=write)

In [16]:
def s3_to_tile(input_tif, item, tile):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine("-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW"))
        
    x_min, y_min, x_max, y_max = tile.tile.bounds
    

    output_tile_name = '{}_L{}_C{}_R{}'.format(item.id,
                                                    tile.level,
                                                    tile.col,
                                                    tile.row)
    gdal.Translate('tmp_{}.tif'.format(output_tile_name),
                   input_tif,
                   projWin=[x_min, y_max, x_max, y_min],
                   projWinSRS='EPSG:4326',
                   xRes=0.002934003239030, 
                   yRes=0.002934003239030,
                   resampleAlg='bilinear')

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name),
                   gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    ds = None
    
    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name))
    ds = gdal.Translate('{}.tif'.format(output_tile_name), ds, options=translate_options)
    ds = None

    band_names = ['NDVI', 'OGVI', 'OTCI', 'Land mask', 'Cloud mask', 'OGVI fail mask']


    ###Re-open the tif file to set noData for 'NDVI', 'OGVI', 'OTCI'
    
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    for index in range(ds.RasterCount):

        srcband = ds.GetRasterBand(index+1)
        srcband.SetDescription(band_names[index])

        if index == 0:
            srcband.SetNoDataValue(255)
    
    ds.FlushCache()
    ds = None
    
    ###Drop generated tiles where land is less than Threshold=0.01
    
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    
    # on borders mask and cloud-mask jumble non-binary values 
    # re-opening them as intgers renders them binary
    
    land_mask=np.array(ds.GetRasterBand(4).ReadAsArray(), dtype= np.uint8)
    cloud_mask=np.array(ds.GetRasterBand(5).ReadAsArray(), dtype= np.uint8)
    
    if np.count_nonzero(land_mask)/land_mask.size < 0.01 : 
        
        os.remove('{}.tif'.format(output_tile_name))
        print('removed {}.tif\n'.format(output_tile_name))
        print('land percentage was {}\n ********\n '.format(np.count_nonzero(land_mask)/land_mask.size))
    else: 
        ndvi_tmp=ds.GetRasterBand(1).ReadAsArray()
        ogvi_tmp=ds.GetRasterBand(2).ReadAsArray()
        otci_tmp=ds.GetRasterBand(3).ReadAsArray()
        # If land_mask=0 set data to noData
    
        masked_data = lambda x,y,z,min_value,max_value,: 255 if y==0  or z==1 or x>max_value or x<min_value else x
        vfunc_masked = np.vectorize(masked_data, otypes=[np.float])
    
    
        updated_ndvi_data = vfunc_masked(ndvi_tmp, land_mask, cloud_mask,-1,1)
        updated_ogvi_data = vfunc_masked(ogvi_tmp, land_mask, cloud_mask,-1,1)
        updated_otci_data = vfunc_masked(otci_tmp, land_mask, cloud_mask,0,6.5)
    
        ds.GetRasterBand(1).WriteArray(updated_ndvi_data)
        ds.GetRasterBand(2).WriteArray(updated_ogvi_data)
        ds.GetRasterBand(3).WriteArray(updated_otci_data)
        ds.GetRasterBand(4).WriteArray(land_mask)
        ds.GetRasterBand(5).WriteArray(cloud_mask)

        with open(output_tile_name + '.properties', 'w') as file:

                file.write('title=Tile L:{1} C:{2} R:{3} {0}\n'.format(item.id,
                                                                  tile.level, 
                                                                  tile.col, 
                                                                  tile.row))

                date='{}/{}'.format(item.datetime.strftime('%Y-%m-%dT%H:%M:%SZ'), 
                                             item.datetime.strftime('%Y-%m-%dT%H:%M:%SZ'))

                file.write('date={}\n'.format(date))

                file.write('geometry={0}'.format(tile.tile.wkt))

    for f in ['tmp_{}.tif'.format(output_tile_name), 'tmp_{}.tif.ovr'.format(output_tile_name)]:

            if os.path.exists(f):

                os.remove(f)

    ds = None
    return True
    

### Tiling

In [17]:
bands = [os.path.join('s3_olci.data', '{}.img'.format(band)) for band in ['OGVI', 'OTCI', 'RC681', 'RC865', 'LQSF']]
        
s3_data = read_s3(bands)
        

In [18]:
ds = gdal.Open(bands[0])

geo_transform = ds.GetGeoTransform()
projection_ref = ds.GetProjectionRef()
    

In [19]:
ogvi = s3_data[:,:,0]
otci= s3_data[:,:,1]
red = s3_data[:,:,2]
nir = s3_data[:,:,3]
lqsf = s3_data[:,:,4]
#otci_quality_flags=s3_data[:,:,5]

In [20]:
mask = get_mask('LAND', lqsf)

In [21]:
cloud_mask =  get_mask('CLOUD', lqsf)

In [22]:
ogvi_fail_mask =  get_mask('OGVI_FAIL', lqsf)
#if mask=1 then ogvi=255

In [23]:
#Calculate NDVI
ndvi_lambda = lambda x,y,z,w,v: 255 if(x+y)==0 or z or w or v==0 else  (x-y)/float(x+y)
vfunc_ndvi = np.vectorize(ndvi_lambda, otypes=[np.float])

ndvi=vfunc_ndvi(nir, red, ogvi_fail_mask ,cloud_mask , mask)


In [24]:
output_name = 'temp.tif'

In [25]:
driver = gdal.GetDriverByName('GTiff')

output = driver.Create(output_name, 
                       ogvi.shape[1], 
                       ogvi.shape[0], 
                       6, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(projection_ref)
output.GetRasterBand(1).WriteArray(ndvi)
output.GetRasterBand(2).WriteArray(ogvi)
output.GetRasterBand(3).WriteArray(otci)
output.GetRasterBand(4).WriteArray(mask)
output.GetRasterBand(5).WriteArray(cloud_mask)
output.GetRasterBand(6).WriteArray(ogvi_fail_mask)

output.FlushCache()

output = None

In [26]:
tiles = s3_tiles(shape(item.geometry), int(tiling_level['value']))


In [27]:
tiles

,col,row,level,s3_tile,tile
0,35,16,5,"POLYGON ((19.71151 0.00000, 20.26610 2.51012, ...","POLYGON ((22.50000 0.00000, 22.50000 5.62500, ..."
1,36,16,5,"POLYGON ((28.12500 0.00000, 22.50000 0.00000, ...","POLYGON ((28.12500 0.00000, 28.12500 5.62500, ..."
2,35,17,5,"POLYGON ((20.93036 5.62500, 21.37750 7.77389, ...","POLYGON ((22.50000 5.62500, 22.50000 11.25000,..."
3,36,17,5,"POLYGON ((22.50000 10.29030, 22.53430 10.28350...","POLYGON ((28.12500 5.62500, 28.12500 11.25000,..."
4,37,17,5,"POLYGON ((28.12500 9.13097, 28.69560 9.00718, ...","POLYGON ((33.75000 5.62500, 33.75000 11.25000,..."
5,37,15,5,"POLYGON ((31.20270 -2.71122, 30.59250 -2.57559...","POLYGON ((33.75000 -5.62500, 33.75000 0.00000,..."
6,36,15,5,"POLYGON ((28.12500 -2.02324, 27.56510 -1.89755...","POLYGON ((28.12500 -5.62500, 28.12500 0.00000,..."
7,35,15,5,"POLYGON ((22.50000 -0.75603, 22.11240 -0.66795...","POLYGON ((22.50000 -5.62500, 22.50000 0.00000,..."
8,37,16,5,"POLYGON ((33.06729 5.62500, 32.98160 5.25293, ...","POLYGON ((33.75000 0.00000, 33.75000 5.62500, ..."


In [28]:
for index, tile in tiles.iterrows():
    
    logging.info('Tile L{} C{} R{}'.format(tile.level,
                                       tile.col,
                                       tile.row))


    s3_to_tile(output_name, item, tile)

removed S3A_OL_2_LFR____20190101T080250_20190101T080550_20190102T125100_0179_039_363_2880_LN1_O_NT_002_L5_C36_R15.tif

land percentage was 0.006287536848051737
 ********
 
removed S3A_OL_2_LFR____20190101T080250_20190101T080550_20190102T125100_0179_039_363_2880_LN1_O_NT_002_L5_C35_R15.tif

land percentage was 0.000858801449513169
 ********
 


In [ ]:
logging.info('Clean-up') 
os.remove(output_name)

shutil.rmtree('s3_olci.data')
os.remove('s3_olci.dim')

time.sleep(45)

for f in glob.glob('./*.tif.aux.xml'):

    os.remove(f)


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.